In [1]:
import asyncio
def go_loop(t=0.001):
    def wrapper(func):
        # @wraps(func)
        async def wrapped(*args, **kwargs):
            while 1:
                await asyncio.sleep(t)
                ret=await func(*args, **kwargs)
                if ret:break
            return
        return wrapped
    return wrapper

def to_pop(li):
    def poper(ret_len=5):
        p=li[:ret_len]
        li.pop(0)
        return p
    return poper

def add_num(func):
    num=-1
    def go_num():
        nonlocal num
        num+=1
        # r=num,*func()
        # incompatible
        r=[num]
        r.extend(func())
        return r
    return go_num

class Buff(list):
    def __init__(self, iterable=(),maxlen=128):
        self.maxlen=maxlen
        assert len(iterable)<=maxlen
        super().__init__(iterable)
    def extend(self):
        raise NotImplementedError
    
    def append(self, item):
        if len(self)>=self.maxlen:
            self.pop(0)
        super().append(item)

In [2]:

import struct
def spliter(format,buffer):
    data=buffer[:struct.calcsize(format)]
    buffer=buffer[struct.calcsize(format):]
    ret=struct.unpack(format,data)
    if len(ret)==1:ret=ret[0]
    return ret,buffer

In [3]:
import sys
import paho.mqtt.client as mqtt

def on_connect(client, userdata, flags, rc):
    if rc != 0:
        print("Connection failed with code:", rc)
        # sys.exit(1)
    else:
        print("Connected successfully!")

def on_message(client, userdata, message):
    print("Message received:", str(message.payload.decode()))

In [4]:
import time

class MQTT_Connection:
    def __init__(self,mqtt_server_port,sub0):
        import socket
        # if mqtt_id is None:
            # mqtt_id=""
        client = mqtt.Client()
        client.username_pw_set("tom", "xiao")
        client.on_connect = on_connect
        client.on_message = on_message
        
        # client.connect("10.144.113.5",1885)
        client.connect(*mqtt_server_port)
        client.loop_start()
        
        client.subscribe(sub0)

        self.s=client
        self.to="ret"
    

    def send(self,data):
        self.s.publish(self.to,data)

In [5]:
def exer(R,payload):
    if b"ret" in payload:
        print("using 'ret' causes problem, skipping")
        return True
    
    ret=[]
    to_ret=False
    if not b"=" in payload:
        payload=b"ret.append("+payload+b")"
        to_ret=True
        
    try:
        exec(payload)
        if to_ret:
            try:
                g=ret[0]
                try:
                    R.send_ret(g)
                except:
                    R.send_ret(str(g))
            except IndexError:g=None
    except Exception as e:
        R.send_ret("error"+str(e))
        return True
    
    return True

In [6]:
class Responder(MQTT_Connection):
    def __init__(self, *args,send_rate=100,):
        super().__init__(*args,)
        self.encode_list=[]
        self.decode_list=[]
        self.encode_list.append(["4B","get_s_time"])
        # self.encode_list.append(["b","get_rssi"])

        self.decode_list.append(["4B","rec_s_time"])
        # ESP不支持assignment

        self.mode=0
        
        self.ret_list=Buff(maxlen=10)

        self.s.on_message=self.on_message


        
    def add_control_set(self,lis):
        self.decode_list.append(lis)
    def add_recv_set(self,lis):
        self.encode_list.append(lis)
        
        
    last_s_time=None
    s_number=0
    def get_s_time(self):
        s_time=time.localtime()[3:6]
        if s_time!=self.last_s_time:
            s_number=0
            last_s_time=s_time
        else:s_number+=1
        ret=*s_time,s_number
        return ret

    
    #ESP不支持assignment
    dict={}
    def decode(self,buffer):
        for form,name in self.decode_list:
            data,buffer=spliter(form,buffer)
            
            #不支持drop掉比最新packet旧的packet
            try:
                if hasattr(self, name):
                    if type(data)==tuple:
                        getattr(self, name)(*data)
                    else:getattr(self, name)(data)
                else:
                    self.dict[name]=data
            except Exception as e:
                print(form)
                print(data)
                print(e)


    def encode(self):
        buffer=b""
        for form, name in self.encode_list:
            if name in self.__dict__ and not callable(self.__dict__[name]):
                data = self.__dict__[name]
            elif hasattr(self, name):
                data = getattr(self, name)()
            try:

                if type(data) in [tuple, list]:
                    buffer += struct.pack(form, *data)
                else:
                    buffer += struct.pack(form, data)
            except Exception as e:
                print(form)
                print(data)
                print(e)
        return buffer

    def send_ret(self,data,header=b"R"):
        import json
        data=json.dumps(data).encode()
        super().send(header+data)
        
    
    def send_exe(self,data,header=b"C"):
        if type(data)==str:data=data.encode()
        super().send(header+data)
        
    def send_m(self,header=b"M"):
        if header in b"MF":
            super().send(header+self.encode())
        
    @go_loop(0.01)         
    async def loop_send(self):
        if len(self.to)==0:return

        self.send_m()
            
    
    # @go_loop(0.009)
    def on_message(self,client, userdata, message):
        # print("Message received:", str(message.payload.decode()))
        data=message.payload
        if not data:return
        
        header=data[0]
        data=data[1:]
        if header in b"MF":
            self.decode(data)
        if header in b"C":
            exer(self,data)
        if header in b"R":
            import json
            data=data.decode()
            try:
                data=json.loads(data)
            except:pass
            self.ret_list.append(data)
        

In [7]:
if not "__file__" in locals():
    R=Responder(("100.64.41.3",1885),"ho0")
    R.to="ho1"

C:\Users\tomxi\AppData\Local\Temp\ipykernel_18472\1161333552.py:8: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Connected successfully!
Message received: Cprint('hihi')
hihi
